In [1]:
from dotenv import load_dotenv
import os


from langchain_community.graphs import Neo4jGraph
from langchain_groq import ChatGroq

In [2]:
load_dotenv('../.env.studiomac')

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")       
NEO4J_DATABASE = os.getenv("NEO4J_DATABASE")

In [3]:
cypher = """
    MATCH(n)
    RETURN count(n)
    """